In [1]:
import sys, os
import glob, re
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
import langdetect
from tqdm import tqdm_notebook

In [2]:
lang_pair = 'en-cs_fr'
split2splitname = {
    'test': 'test_2016_flickr',
    'test1': 'test_2017_flickr',
    'test2': 'test_2017_mscoco',
    'test3': 'test_2018_flickr',
}

In [3]:
text_ckpt = f'./../checkpoint/full/{lang_pair}/vanilla/transformer_tiny/'
imag_ckpt = f'./../checkpoint/zero-shot/{lang_pair}/detr_resnet-50-dc5/imagination-lvp_static/'

src_files = {
    k: v for k, v in 
    [(k, f'./../data/{lang_pair}/tok/{v}.en-fr.en') for k, v in split2splitname.items()]
    if os.path.exists(v)
}

ref_files = {
    k: v for k, v in 
    [(k, f'./../data/{lang_pair}/tok/{v}.en-fr.fr') for k, v in split2splitname.items()]
    if os.path.exists(v)
}

img_files = {
    k: v for k, v in 
    [(k, f'./../data/{lang_pair}/image_splits/{v}.en-fr.txt') for k, v in split2splitname.items()]
    if os.path.exists(v)
}

src_files, ref_files, img_files

({'test': './../data/en-cs_fr/tok/test_2016_flickr.en-fr.en',
  'test1': './../data/en-cs_fr/tok/test_2017_flickr.en-fr.en',
  'test2': './../data/en-cs_fr/tok/test_2017_mscoco.en-fr.en',
  'test3': './../data/en-cs_fr/tok/test_2018_flickr.en-fr.en'},
 {'test': './../data/en-cs_fr/tok/test_2016_flickr.en-fr.fr',
  'test1': './../data/en-cs_fr/tok/test_2017_flickr.en-fr.fr',
  'test2': './../data/en-cs_fr/tok/test_2017_mscoco.en-fr.fr',
  'test3': './../data/en-cs_fr/tok/test_2018_flickr.en-fr.fr'},
 {'test': './../data/en-cs_fr/image_splits/test_2016_flickr.en-fr.txt',
  'test1': './../data/en-cs_fr/image_splits/test_2017_flickr.en-fr.txt',
  'test2': './../data/en-cs_fr/image_splits/test_2017_mscoco.en-fr.txt',
  'test3': './../data/en-cs_fr/image_splits/test_2018_flickr.en-fr.txt'})

In [4]:
text_hyp_files = list(glob.glob(text_ckpt + '*/*.en-fr.fr.hyp', recursive=True))
imag_hyp_files = [fn for fn in glob.glob(imag_ckpt + '*/*.en-fr.fr.hyp', recursive=True) if ('incongruent' not in fn and 'replaced_all' not in fn)]

len(text_hyp_files), len(imag_hyp_files), imag_hyp_files

(21,
 39,
 ['./../checkpoint/zero-shot/en-cs_fr/detr_resnet-50-dc5/imagination-lvp_static/1442/valid.en-fr.fr.hyp',
  './../checkpoint/zero-shot/en-cs_fr/detr_resnet-50-dc5/imagination-lvp_static/1442/test.en-fr.fr.hyp',
  './../checkpoint/zero-shot/en-cs_fr/detr_resnet-50-dc5/imagination-lvp_static/1442/test1.en-fr.fr.hyp',
  './../checkpoint/zero-shot/en-cs_fr/detr_resnet-50-dc5/imagination-lvp_static/1442/test2.en-fr.fr.hyp',
  './../checkpoint/zero-shot/en-cs_fr/detr_resnet-50-dc5/imagination-lvp_static/1442/test3.en-fr.fr.hyp',
  './../checkpoint/zero-shot/en-cs_fr/detr_resnet-50-dc5/imagination-lvp_static/1442/test4.en-fr.fr.hyp',
  './../checkpoint/zero-shot/en-cs_fr/detr_resnet-50-dc5/imagination-lvp_static/1442/test5.en-fr.fr.hyp',
  './../checkpoint/zero-shot/en-cs_fr/detr_resnet-50-dc5/imagination-lvp_static/1442/test-replace_all.en-fr.fr.hyp',
  './../checkpoint/zero-shot/en-cs_fr/detr_resnet-50-dc5/imagination-lvp_static/1442/test1-replace_all.en-fr.fr.hyp',
  './../checkp

In [5]:
re_index = re.compile('H-(\d+)')
def load_hyp(fn):
    def is_hyp(s):
        return s.startswith('H')
    
    def find_hyp(s):
        index = re_index.findall(s)[0]
        hyp = s.strip().split('\t')[2]
        return int(index), hyp
    
    def sort_hyp(l):
        return [i[1] for i in sorted(l, key=lambda _: _[0])]
    
    return sort_hyp([find_hyp(s) for s in open(fn, 'r') if is_hyp(s)])

load_hyp(imag_hyp_files[0])[:5]

['un groupe d&apos; hommes chargent du coton sur un camion .',
 'un homme dormant dans une pièce verte sur un canapé .',
 'un garçon portant des écouteurs est assis sur les épaules d&apos; une femme .',
 'deux hommes installant une hutte de pêche bleue sur un lac .',
 'un homme dégarni vêtu d&apos; un gilet de sauvetage rouge est assis dans un petit bateau .']

In [6]:
def load_ref(fn):
    return [l.strip() for l in open(fn, 'r')]

load_ref(ref_files['test'])[:5]

['un homme avec un chapeau orange regardant quelque chose .',
 'un terrier de boston court sur l&apos; herbe verdoyante devant une clôture blanche .',
 'une fille en tenue de karaté brisant un bâton avec un coup de pied .',
 'cinq personnes avec des vestes d&apos; hiver et des casques sont debout dans la neige , avec des motoneiges en arrière-plan .',
 'des gens réparent le toit d&apos; une maison .']

In [7]:
def compute_sent_bleu(ref, hyp):
    return [
        (sentence_bleu([r], h), r, h) for r, h in zip(ref, hyp)
    ]

compute_sent_bleu(load_ref(ref_files['test'])[:5], load_hyp(imag_hyp_files[0])[:5])

/home/tosho/.pyenv/versions/anaconda3-2022.05/envs/fairseq4mm/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


[(0.28763290795903784,
  'un homme avec un chapeau orange regardant quelque chose .',
  'un groupe d&apos; hommes chargent du coton sur un camion .'),
 (0.21401098385318648,
  'un terrier de boston court sur l&apos; herbe verdoyante devant une clôture blanche .',
  'un homme dormant dans une pièce verte sur un canapé .'),
 (0.189767906043468,
  'une fille en tenue de karaté brisant un bâton avec un coup de pied .',
  'un garçon portant des écouteurs est assis sur les épaules d&apos; une femme .'),
 (1.7349120493717766e-78,
  'cinq personnes avec des vestes d&apos; hiver et des casques sont debout dans la neige , avec des motoneiges en arrière-plan .',
  'deux hommes installant une hutte de pêche bleue sur un lac .'),
 (0.2119035164974129,
  'des gens réparent le toit d&apos; une maison .',
  'un homme dégarni vêtu d&apos; un gilet de sauvetage rouge est assis dans un petit bateau .')]

In [8]:
def load_all_hyps(hyps):
    hyp_dict = {} # split -> seed -> hyp
    
    def get_split(fn):
        return fn.split('/')[-1].split('.')[0]
    def get_seed(fn):
        return fn.split('/')[-2]
    
    for hyp in hyps:
        split = get_split(hyp)
        seed = get_seed(hyp)
        
        if split not in hyp_dict:
            hyp_dict[split] = {}
        
        hyp_dict[split][seed] = load_hyp(hyp)
        
    return hyp_dict

text_hyps = load_all_hyps(text_hyp_files)
imag_hyps = load_all_hyps(imag_hyp_files)

In [9]:
def load_all_ref(ref_files):
    return {
        split: load_ref(fn)
        for split, fn in ref_files.items()
    }

srcs = load_all_ref(src_files)
refs = load_all_ref(ref_files)
imgs = load_all_ref(img_files)

In [10]:
def disp_scores(key, hyps, srcs, refs, images, fp=None):
    for split in hyps.keys():
        if split not in refs:
            continue

        for seed in hyps[split].keys():
            for i, (src, ref, img, hyp) in tqdm_notebook(enumerate(zip(srcs[split], refs[split], images[split], hyps[split][seed]))):
                print('\t'.join(map(str, [
                    key,
                    split2splitname[split],
                    seed,
                    i + 1,
                    img,
                    src,
                    ref,
                    hyp,
                    sentence_bleu([ref], hyp)*100,
                    meteor_score([ref.split()], hyp.split())*100,
                    langdetect.detect(hyp),
                ])), file=fp)

# with open(f'./../sent_results-{lang_pair}.txt', 'w') as fp:
#     disp_scores('text-only', text_hyps, srcs, refs, imgs, fp=fp)
#     disp_scores('imagination', imag_hyps, srcs, refs, imgs, fp=fp)

/tmp/ipykernel_3576554/1348394971.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, (src, ref, img, hyp) in tqdm_notebook(enumerate(zip(srcs[split], refs[split], images[split], hyps[split][seed]))):


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]